In [ ]:
!pip install resemblyzer
!pip install pydub
!pip install spectralcluster
# !pip install links_cluster
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 625 kB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
     |████████████████████████████████| 66 kB 6.1 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72383 sha256=4cc25e0dbc3878848b626186fbe3eee324bab441eb5863b88415e9febb51fea1
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26325 sha256=51ffe20914309f6a872ce5869fd9c5ceeb44b93e98fb81d4e31875506f317d65
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built webrtcvad typing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import links_cluster

In [ ]:
!pip install pydub
!pip install audioread
!pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.3 MB/s 


In [ ]:
import librosa
import soundfile as sf

In [ ]:
# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    chunksList = []
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                # print("Error:", str(e))
                pass
            else:
                text = f"{text.capitalize()}. "
                # print(chunk_filename, ":", text)
                chunksList.append(text)
                whole_text += text
    # return the text for all chunks detected
    return (whole_text, chunksList)

In [ ]:
def byte_to_float(byte):
    # byte -> int16(PCM_16) -> float32
    return pcm2float(np.frombuffer(byte,dtype=np.int8), dtype='float32')

def pcm2float(sig, dtype='float32'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max

In [ ]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment
import numpy as np
import contextlib
import librosa
import struct
import soundfile

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""
def record(sec=10):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  # audio = AudioSegment.from_file(BytesIO(b))
  # print(BytesIO(b).getvalue())
  someByte = bytearray()
  someByte.extend(BytesIO(b).getvalue())
  # data.extend(BytesIO(b).getvalue())
  # print(data)
  # byteData = b''.join(data)
  # print(byteData)
  with open('audioLive.wav','wb') as f:
    f.write(bytes(BytesIO(b).getvalue()))
  return (someByte, bytes(someByte))

In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from pydub.utils import mediainfo

"""
Embeddings class has following abilities:
1. Find the Speech Segments with existence of voice
2. Obtain the d-vector embeddings for each segment
3. Perform Clustering over the obtained d-vector embeddings for the speech segments
4. Find the speaker labels with start and stop time from the audio file.

The class Embeddings has the following parameters being calculated for an audio file while instantiating:
1. Sampling rate (sampling_rate)
2. Wave Splits of the Speech Segments (wav_splits)
3. D-vector embeddings (cont_embeds)

Functions:
1. Constructor:
def __init__(self, audio_file_path) -> Specify the path of audio file to create an Embeddings object for a single file.
2. Voice Activity Detection:
def VoiceActivityDetection(self) -> Returns the Wave splits of speech segments (wav_splits)
3. Embeddings Extraction (d-vectors):
def ExtractEmbeddings(self) -> Returns the d-vector embeddings for the speech segments
4. Clustering:
def OfflineSpectralClustering(self) -> Returns labels, i.e., the speaker identities of the audio file
5. Labelings of Speakers:
def SpeakerLabeling(self, label) -> Returns a tuple of (label, start time, stop time) for each segment obtained by VoiceActivityDetection (through wav splits)

"""

class Embeddings:
  def __init__(self, audio_file_path, data_flag, Data=None):
    #Technical info (title,date,duration,sample rate,channels,language,etc) of file is stored in variable info using open source mediainfo library
    info = mediainfo(audio_file_path)
    #Sample rate column from info table is stored in sampling rate
    self.sampling_rate = info['sample_rate']
    self.audio_file_path = audio_file_path
  
  #    Here we are using open source library called resemblyzer which is used for 2 tasks: 
  #    1. Speech Detection and Segmentation
  #    2. Embeddings Extraction
  #    To instantiate Embeddings object, pass the parameter audio_file_path.
    self.data = Data
    #Path name of the .wav file is retrieved and stored in wav_fpath
    wav_fpath = Path(audio_file_path)

  #    Used preprocess_wav() which internally uses a VAD to trim out the silences in the audio file. 
  #    It is also used to remove outliers present in our speech data and also to normalize the decibel level of audio. 
  #    Function will take input as our file path and output of that function will give us the processed Audio file.   
    if data_flag==0:
      wav = preprocess_wav(Data)
    else:
      wav = preprocess_wav(wav_fpath)

  #   Then we create an instance of the VoiceEncoder class as encoder, by passing cpu as default device. 
  #   VoiceEncoder is deep learning model which is used to derive high-level representation of voice.  
    self.encoder = VoiceEncoder("cpu")

  #   There is embed_utterance function availabel. This function takes the processed wav file. 
  #   It will create segments of voice data as wav_splits and as well makes MFCCs of these segments.
  #   Finally creates embeds of these audio segments. And stored in cont_embeds. 
    _, cont_embeds, wav_splits = self.encoder.embed_utterance(wav, return_partials=True, rate= 4)

    #start and end time of each window for which a d-vector has been created is stored in wav_splits
    self.wav_splits = wav_splits

  # The cont_embeds is a N by D matrix, where N is the number of segments created (which is equal to the number of d-vectors) 
  #  and D is the dimension of each d-vector, which by default is 256. 
  #  wav_splits is a list with the start and end time of each window for which a d-vector has been created.    
    self.cont_embeds = cont_embeds

  # In Voice Activity Detection the presence or absence of speech in an audio segment is detected.
  #    1. It is done by taking the audio segments which are split into segments of 5-40ms duration as an input.
  #    2. Then two features namely signal energy and spectral centroid are extracted and compared to the threshold value.
  #    3. Then depending upon the value, the decision about whether or not speech is present is taken.
  #    4. The wav_splits containing only speech part are returned
  def VoiceActivityDetection(self):
    return self.wav_splits

 # this function will return the embeddings generated while instantiating.
  def ExtractEmbeddings(self):
    return self.cont_embeds
  
  #  This function is used to perform spectral offline clustering on our embeddings. we use open source module called SpectralCluster.
  #  We can build our own Spectral clustering model using SpectralCluster class. we can set different parameters such as minimum cluster and 
  #  maximum cluster for our model. In our case we set minimum of 3 and maximum of 100 clusters that our model will produce.
  def OfflineSpectralClustering(self, clusterer, vector):
    from links_cluster import LinksCluster
    predicted_cluster = clusterer.predict(vector)
    return predicted_cluster

  def SpeakerLabeling(self, labels):
    # the number of samples per second is given by sampling_rate
    currentLabel = -1
    aggregatedLabels = []
    for i in range(len(labels)):
      if currentLabel!=labels[i]:
        aggregatedLabels.append(labels[i])
        currentLabel = labels[i]
      else:
        continue
    return aggregatedLabels

  def update(self, data):
    self.data = data
    wav = preprocess_wav(self.data)
    _, cont_embeds, wav_splits = self.encoder.embed_utterance(wav, return_partials=True, rate= 4)
    self.wav_splits = wav_splits
    self.cont_embeds = cont_embeds

In [ ]:
# This function is the step by step implementation of all the processes which we have defined in Embeddings Class above. 

def SpeakerDiarization(embedding_obj, clusterer):
   # Create the embedding_obj object of Embedding class.
   # Returns the speech segments.
  # detectedVoiceSegments = embedding_obj.VoiceActivityDetection()
   # Obtain embeddings by calling the function ExtractEmbeddings().
  embeddings = embedding_obj.ExtractEmbeddings()
   # Apply spectral offline clustering to embeddings to get speaker labels using OfflineSpectralClustering() function.
  clusteredLabels = []
  print(embeddings.shape)
  for i in range(len(embeddings)):
    clusteredLabels.append(embedding_obj.OfflineSpectralClustering(clusterer, embeddings[i]))
  labels = embedding_obj.SpeakerLabeling(clusteredLabels)
  return labels

In [ ]:
def getMappedTranscript(currentStatesList, chunksList):
  transcriptMap = []
  if len(currentStatesList)>0 and len(chunksList)>0:
    iteration = max(len(currentStatesList), len(chunksList))
    i = 0
    temp = currentStatesList[0]
    while i!=iteration:
      if i<len(currentStatesList):
        if i<len(chunksList):
          transcriptMap.append([currentStatesList[i], chunksList[i]])
          i += 1
          temp = currentStatesList[i]
          continue
      else:
        transcriptMap.append([temp, chunksList[i]])
        i += 1
        continue
      i += 1
  return transcriptMap

In [ ]:
dataframe = {"Speaker": [], "Text": []}

In [ ]:
from links_cluster import LinksCluster
def SpeakerDiarizationAndSTT(audioLength):
  duration = 20
  count = 10
  ByteData = bytearray()
  fullFloatData = []
  flag = 0
  currentStatesStartIndex = 0
  print("Recording Started...")
  while(count!=0):
    ByteData, data = record(duration)
    x,_ = librosa.load('audioLive.wav')
    sf.write('audioLive_.wav', x, 20500)
    
    FloatData = byte_to_float(bytes(ByteData))
    fullFloatData.extend(FloatData)

    if flag==0:
    #   embedding_obj = Embeddings('audioLive_.wav', 0, np.array(fullFloatData))
      links_cluster = LinksCluster(0.95, 0.7, 1.0, store_vectors=True)
      flag = 1
    # else:
    #   embedding_obj.update(np.array(FloatData))

    embedding_obj = Embeddings('audioLive_.wav', 1)

    currentStatesList = SpeakerDiarization(embedding_obj, links_cluster)
    transcriptObtained, chunksList = get_large_audio_transcription('audioLive_.wav')
    currentStatesStartIndex = len(currentStatesList)
    mappedTranscript = getMappedTranscript(currentStatesList, chunksList)

    # Adding the obtained transcript text and the speakers identities to the Dataframe
    print("Writing Data...")
    for it in range(len(mappedTranscript)):
      dataframe['Speaker'].append(mappedTranscript[it][0])
      dataframe['Text'].append(mappedTranscript[it][1])

    print(mappedTranscript)
    # print(currentStatesList)
    # print(chunksList)
    count-=1

In [ ]:
SpeakerDiarizationAndSTT(200)

Recording Started...


<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Loaded the voice encoder model on cpu in 0.01 seconds.
(70, 256)
Writing Data...
[[0, "I don't want to watch a country deep space 9 is better. "], [1, 'How big is deep space nine better than saturn 3. '], [2, 'Simple fraction.. '], [3, 'Please watch baby alive. '], [1, 'In what sense is that a compromise. '], [3, 'Fire. ']]


<IPython.core.display.Javascript object>

...

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Loaded the voice encoder model on cpu in 0.01 seconds.
(74, 256)
Writing Data...
[[1, 'Government projects on distant planets. '], [3, 'Tim hortons. '], [6, 'You know. '], [1, "It's getting late. "], [3, "So do i get to drive these things are what i'm sorry but something's come up. "]]


<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Loaded the voice encoder model on cpu in 0.01 seconds.
(68, 256)
Writing Data...
[[1, 'Are you a scientist lahore. '], [3, 'No one scientist like howard. '], [1, 'My mother is going to love her. '], [7, 'Oh nice. '], [1, 'Maybe we can carpool when they visit you in federal prison. '], [1, "I'm sorry i totally interrupted. "]]


In [ ]:
import pandas as pd
print(pd.DataFrame(dataframe))

     Speaker                                               Text
0          0                                       73 east on. 
1          1                             I don't want to work. 
2          0                                  Best 90s better. 
3          1      How is deep space nine better than saturn 3. 
4          2                                 It sticks better. 
..       ...                                                ...
103        3                     No one scientist like howard. 
104        1                   My mother is going to love her. 
105        7                                          Oh nice. 
106        1  Maybe we can carpool when they visit you in fe...
107        1                  I'm sorry i totally interrupted. 

[108 rows x 2 columns]


In [ ]:
df = pd.DataFrame(dataframe)
df.to_excel('data_1.xlsx')